# Implementation of new curtailment constraints in MESSAGEix-GLOBIOM

Load modules:

In [2]:
import message_ix
import ixmp as ix
from message_ix.utils import make_df
from itertools import product
from timeit import default_timer as timer
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

<IPython.core.display.Javascript object>

### Steps taken:
1. Read curtailment parameters derived from PyPSA-Eur
2. Load scenario in MESSAGEix-GLOBIOM
3. Technology linkage (adding SDES and LDES)
4. Add curtailment technologies and relations to the scenario
5. Solve scenario

### Step 1) fetch curtailment parameters

Read data and define dictionaries:

In [4]:
# gamma coefficients (marginal curtailment rates)
path = "parameters/"
gamma_ij_wind = pd.read_csv(path + "gamma_ij_wind_primary.csv",index_col=0).dropna()
gamma_ij_solar = pd.read_csv(path + "gamma_ij_solar_primary.csv",index_col=0).dropna()

# beta coefficients (counter acting term from renewable integration support measures)
techs = ["LDES","SDES"] 
beta_tech_wind = {}
beta_tech_solar = {}
beta_tech_aggr = {}
for tech in techs:
    beta_tech_wind[tech] = pd.read_csv(path + "beta_"+tech+"_wind.csv",index_col=0).dropna()
    beta_tech_solar[tech] = pd.read_csv(path + "beta_"+tech+"_solar.csv",index_col=0).dropna()

# renewable shares
wind_shares_pypsa = pd.read_csv(path + "wind_shares.csv",index_col=0).loc[gamma_ij_wind.index]
solar_shares_pypsa = pd.read_csv(path + "solar_shares.csv",index_col=0).loc[gamma_ij_solar.index]

# create dictionaries that contain the parameters
gamma_dict = gamma_ij_solar.to_dict()["0"]
gamma_wind_dict = gamma_ij_wind.to_dict()["0"]
gamma_dict.update(gamma_wind_dict) # dictionary containing all gamma coefficients

beta_dict = {}
for tech in techs:
    beta_dict["wind", tech] = beta_tech_wind[tech].to_dict()["0"]
    beta_dict["solar", tech] = beta_tech_solar[tech].to_dict()["0"]

renewable_penetration_dict = solar_shares_pypsa.to_dict()["0"]
wind_penetration = wind_shares_pypsa.to_dict()["0"]
renewable_penetration_dict.update(wind_penetration)

Initialize curtailment relations:

In [5]:
# number of bins (determined from the PyPSA-Eur data):
bins = list(set(renewable_penetration_dict.values()))
bins.sort()

bins_dict = {}
curt_relation = {} # i = solar, j = wind
curt_relation_tech = {}
new_bins = {}
prefix = "vre" # prefix of relation name
for i in range(len(bins)): # wind index
    for j in range(len(bins)): # solar index
        
        w = "wind_curtailment_w" + str(i) + "s" + str(j) # index naming in the data achieved from PyPSA-Eur
        s = "solar_curtailment_s" + str(j) + "w" + str(i) # index naming in the data achieved from PyPSA-Eur

        wind_curt_name = "wind_curtailment" + str(i+1) # naming of wind curtailment technology in MESSAGEix-GLOBIOM
        solar_curt_name = "solar_curtailment" + str(j+1) # naming of solar curtailment technology in MESSAGEix-GLOBIOM
        # for naming convenience, we shift the index naming by one such that the first bin starts at index = 1

        if (w not in gamma_ij_wind.index) and (s not in gamma_ij_solar.index): 
            continue
        
        LDES_tech_term = beta_dict["wind","LDES"][w] + beta_dict["solar","LDES"][s]
        SDES_tech_term = beta_dict["wind","SDES"][w] + beta_dict["solar","SDES"][s]

        new_bins[prefix + "_curtailment_w" + str(i+1) + "s" + str(j+1)] = [wind_curt_name, solar_curt_name]
        
        curt_relation[prefix + "_curtailment_w" + str(i+1) + "s" + str(j+1)] = [{wind_curt_name:gamma_ij_wind.loc[w].item(),
                                                                                solar_curt_name:gamma_ij_solar.loc[s].item(),
                                                                                "LDES":LDES_tech_term,
                                                                                "SDES":SDES_tech_term,
                                                                                }]
        
        if j == 0:
            curt_relation_tech["wind_curtailment_" + str(i+1)] = wind_curt_name
            bins_dict["wind_curtailment_" + str(i+1)] = bins[i]
        
        if i == 0:
            curt_relation_tech["solar_curtailment_" + str(j+1)] = solar_curt_name
            bins_dict["solar_curtailment_" + str(j+1)] = bins[j]

curt_relation_tech = {k: curt_relation_tech[k] for k in sorted(curt_relation_tech)}

# print first five entries of the dictionaries
print({k: curt_relation[k] for k in list(curt_relation)[:5]})
print({k: curt_relation_tech[k] for k in list(curt_relation_tech)[:5]})

{'vre_curtailment_w1s1': [{'wind_curtailment1': 0.0037371249171338, 'solar_curtailment1': 0.002290072586197, 'LDES': 0.0, 'SDES': 0.0}], 'vre_curtailment_w1s2': [{'wind_curtailment1': 0.0276494369460103, 'solar_curtailment2': 0.2527864681633947, 'LDES': -0.046101324745595196, 'SDES': -0.3050250462972438}], 'vre_curtailment_w1s3': [{'wind_curtailment1': 0.0097918369226165, 'solar_curtailment3': 0.4818535505889985, 'LDES': 0.0400177688060895, 'SDES': -0.2209158072357206}], 'vre_curtailment_w1s4': [{'wind_curtailment1': 0.0010403098686803, 'solar_curtailment4': 0.1559578472451435, 'LDES': -0.07287316007021549, 'SDES': -0.2106788137781615}], 'vre_curtailment_w1s5': [{'wind_curtailment1': -0.001500484208699, 'solar_curtailment5': 0.0701529611080714, 'LDES': -0.1488328376308044, 'SDES': -0.1477049124733644}]}
{'solar_curtailment_1': 'solar_curtailment1', 'solar_curtailment_2': 'solar_curtailment2', 'solar_curtailment_3': 'solar_curtailment3', 'solar_curtailment_4': 'solar_curtailment4', 'sol

### Step 2) load scenario in MESSAGEix-GLOBIOM

In [6]:
# load ixmp database
mp = ix.Platform('local',jvmargs=["-Xmx8G"])
solve_scenario = True # if True, solving scenario at the end
regions = ["R11_WEU"]

# reference scenario: 
sc_ref = message_ix.Scenario(mp, model="MESSAGEix-GLOBIOM", scenario="ENGAGE_SSP2_EN_NPi2020_500",version=1)
model = "MESSAGEix-GLOBIOM"
scen = "ENGAGE_SSP2_EN_NPi2020_500_PyPSA_Eur"

# clone scenario and add modifications
sc = sc_ref.clone(model=model, scenario=scen, keep_solution=False)

sc.check_out()

years = sc.set('year')

### Step 3) Technology linkage

Here, we replace the generic storage power plant "**stor_ppl**" with a short-duration and long-duration electricity storage (SDES and LDES). 

**LDES**
- duration (E/G_d): 50 hrs (variable)
- Efficiency (round-trip): 0.48
- Technology cost: 
    - Charge link: 450 EUR/kW (+ 2% FOM)
    - Storage: 12 EUR/kWh (+ 2% FOM)
    - Discharge link: 500 EUR/kW (+ 2% FOM) 
    
    Aggregated cost: 1/efficiency * 450 EUR/kW + duration * 12 EUR/kWh + 500 EUR/kW = 2,037.5 EUR/kW
- Capacity factor: 20.5% (variable)
- Lifetime: 20 years

**SDES**
- duration (E/G_d): 6 hrs (variable)
- Efficiency (round-trip): 0.94
- Technology cost: 
    - Charge link (battery inverter/converter): 160 EUR/kW
    - Storage: 142 EUR/kWh
    - Discharge link: 0 EUR/kW
    
    Aggregated cost: 1/effficiency * 160 EUR/kW + duration * 142 EUR/kWh + 0 EUR/kW = 1,022 EUR/kW
- Capacity factor: 11.9% (variable)
- Lifetime: 25 years


In MESSAGEix-GLOBIOM, the storage tech needs to have the following attributes defined:
- input electricity (par)
- output (par)
- inv_cost (par)
- technology (set)
- var_cost (par)
- technical_lifetime (par)
- capacity_factor (par)


Storage technology attributes from PyPSA-Eur:

In [41]:
# inputs
efficiency_ldes = 0.48 # round-trip efficiency of LDES
c_c_ldes = 450 # EUR/kW
c_hat_ldes = 12 # EUR/kWh
c_d_ldes = 500 # EUR/kW
ldes_lifetime = 20 # years

efficiency_sdes = 0.94 # round-trip efficiency of LDES
c_c_sdes = 160 # EUR/kW
c_hat_sdes = 142 # EUR/kWh
c_d_sdes = 0 # EUR/kW
sdes_lifetime = 25 # years

# read outputs from PyPSA-Eur concerning storage capacity and operation
path = "calculated_metrics/new_SDES_LDES_co2_lim/"

# duration
duration_ldes = pd.read_csv(path + "LDES_duration.csv",index_col=[0,1]) # LDES duration [hours]
duration_ldes_avg = duration_ldes.mean().item()
duration_sdes = pd.read_csv(path + "SDES_duration.csv",index_col=[0,1]) # SDES duration [hours]
duration_sdes_avg = duration_sdes.mean().item()

# capacity factor
ldes_capacity_factor = pd.read_csv(path + "LDES_capacity_factor.csv",index_col=[0,1]) # LDES CF [%]
ldes_capacity_factor_avg = ldes_capacity_factor[ldes_capacity_factor > 0].mean().item()
sdes_capacity_factor = pd.read_csv(path + "SDES_capacity_factor.csv",index_col=[0,1]) # LDES CF [%]
sdes_capacity_factor_avg = sdes_capacity_factor[sdes_capacity_factor > 0].mean().item()

# other features not used to calculate costs but relevant for comparison:
ldes_no_cycles = pd.read_csv(path + "LDES_no_cycles.csv",index_col=[0,1]) # number of cycles
ldes_no_cycles_avg = ldes_no_cycles.mean().item()
sdes_no_cycles = pd.read_csv(path + "SDES_no_cycles.csv",index_col=[0,1])
sdes_no_cycles_avg = sdes_no_cycles.mean().item()
ldes_dod = pd.read_csv(path + "LDES_dod.csv",index_col=[0,1]) # depth of discharge
ldes_dod_avg = ldes_dod.mean().item()
sdes_dod = pd.read_csv(path + "SDES_dod.csv",index_col=[0,1])
sdes_dod_avg = sdes_dod.mean().item()

# aggregated cost:
ldes_inv_cost = 1/efficiency_ldes*c_c_ldes + duration_ldes*c_hat_ldes + c_d_ldes # EUR/kW
sdes_inv_cost = 1/efficiency_sdes*c_c_sdes + duration_sdes*c_hat_sdes + c_d_sdes # EUR/kW # 1022 # EUR/kW

Define pandas dataframes with storage attributes:

In [183]:
def add_storage_tech(tech, years, capacity_factor, inv_cost, lifetime, efficiency):

    df_input = make_df('input',
                       node_loc="R11_WEU",
                       technology= tech,
                       year_vtg=years,
                       year_act=years,
                       mode="M1",
                       node_origin="R11_WEU",
                       commodity="electr",
                       level="secondary",
                       time="year",
                       time_origin="year",
                       value = efficiency,
                       unit = "GWa")

    df_output = make_df('output',
                            node_loc = "R11_WEU",
                            technology = tech,
                            year_vtg = years,
                            year_act = years,
                            mode = "M1",
                            node_dest = 'R11_WEU',
                            commodity = 'exports',
                            level = "secondary",
                            time = "year",
                            time_dest = "year",
                            value = 1,
                            unit = "GWa",
                            )

    df_CF = make_df('capacity_factor',
                            node_loc = "R11_WEU",
                            technology = tech,
                            year_vtg = years,
                            year_act = years,
                            time = "year",
                            value = capacity_factor,
                            unit = "%",
                            )

    df_inv_cost = make_df('inv_cost',
                            node_loc="R11_WEU",
                            technology=tech,
                            year_vtg=years,
                            value=inv_cost,
                            unit="USD/GWa",
                            )

    df_lifetime = make_df('technical_lifetime',
                            node_loc="R11_WEU",
                            technology=tech,
                            year_vtg=years,
                            value=lifetime,
                            unit="y")
    

    # Now that we have added the LDES and SDES as technologies in the scenario, 
    # we also need to define their parameters in the VRE integration constraints. 
    # Here, we add the parameters for LDES and SDES to the firm capacity constraint
    # and the flexibility constraint. We assume the storage is fully contributing to
    # the firm capacity constraint and the flexibility constraint, i.e., they have 
    # values of 1.0 (which is the same value for stor_ppl):
    
    # firm capacity constraint
    df_res_marg = make_df('relation_total_capacity',
                          relation = "res_marg",
                          node_rel = "R11_WEU",
                          year_rel = years,
                          technology = tech,
                          value = 1.0,
                          unit = "???"
                          )
    
    # flexibility constraint
    df_oper_res = make_df('relation_activity',
                          relation = "oper_res",
                          node_rel = "R11_WEU",
                          year_rel = years,
                          node_loc = "R11_WEU",
                          technology = tech,
                          year_act = years,
                          mode = "M1",
                          value = 1.0,
                          unit = "???"
                          )
    
    sc.add_set("technology", tech)
    sc.add_par('input',df_input)
    sc.add_par('output',df_output)
    sc.add_par("capacity_factor", df_CF)
    sc.add_par('inv_cost',df_inv_cost)
    sc.add_par('technical_lifetime',df_lifetime)
    sc.add_par('relation_total_capacity',df_res_marg)
    sc.add_par("relation_activity",df_oper_res)
    
    # sc.idx_names('capacity_factor')

Add storage technologies to the scenario:

In [184]:
add_storage_tech("LDES", years, ldes_capacity_factor, ldes_inv_cost, ldes_lifetime, efficiency_ldes)
add_storage_tech("SDES", years, sdes_capacity_factor, sdes_inv_cost, sdes_lifetime, efficiency_sdes)

Now that we have added LDES and SDES as technologies and copied the parameters from the firm capacity and flexibility constraints for stor_ppl (only in Western Europe), we will remove "*stor_ppl*" from these two constraints:

In [185]:
# Removing "stor_ppl" from the firm capacity constraint (res_marg)
df_res_marg = sc.par("relation_total_capacity", 
                    {"relation":"res_marg",
                     "technology":"stor_ppl",
                     "node_rel":"R11_WEU"})
sc.remove_par("relation_total_capacity", df_res_marg)

# Removing "stor_ppl" from the flexibility constraint (oper_res)
df_oper_res = sc.par("relation_activity", 
                    {"relation":"oper_res",
                     "technology":"stor_ppl",
                     "node_rel":"R11_WEU"})
sc.remove_par("relation_activity", df_oper_res)

# Remove "stor_ppl" from the integration cost constraint
# ---> "stor_ppl" is not included in this constraint

# Remove "stor_ppl" from the curtailment constraint 
# ---> will be done later 



### Step 4) Add curtailment technologies and relations to the scenario

In [186]:
# Curtailment relations and technologies (for checking)
curtail_relations = [x for x in set(sc.set("relation")) if "curtailment" in x]
curtail_techs = [x for x in set(sc.set("technology")) if "curtailment" in x]

# Add wind/solar curtailment data for new bins (not present in the old data)
new_relations = [x for x in bins_dict.keys() if x not in set(sc.set("relation"))]
new_techs = [curt_relation_tech[x] for x in new_relations]

# Add new set elements to the scenario
sc.add_set("technology", new_techs) # here, new technologies cover the ones for "wind_curtailment_{}" and "solar_curtailment_{}".
sc.add_set("relation", new_relations) # here, new relations cover the ones for "wind_curtailment_{}" and "solar_curtailment_{}".

print("Added new technologies: ", new_techs)
print("Added new relations: ", new_relations)

Added new technologies:  ['solar_curtailment4', 'solar_curtailment5', 'solar_curtailment6', 'solar_curtailment7', 'wind_curtailment4', 'wind_curtailment5', 'wind_curtailment6', 'wind_curtailment7']
Added new relations:  ['solar_curtailment_4', 'solar_curtailment_5', 'solar_curtailment_6', 'solar_curtailment_7', 'wind_curtailment_4', 'wind_curtailment_5', 'wind_curtailment_6', 'wind_curtailment_7']


In [187]:
# In this step, we add the parameters representing the base curtailment, i.e., we don't yet 
# consider the role of curtailment-reducing technologies like storage etc.
parname = "relation_activity"
for rel, region in product(sorted(bins_dict.keys()), regions):
    # Load existing data (and use it later for configuring the new data)
    old = sc_ref.par(parname, {"node_loc": region, 
                               "relation": rel})

    # If there is no data for this bin, use the data of the previous bin
    if old.empty:
        rel_pre = ("").join(
            [rel.split("_curt")[0],
             "_curtailment_",
             str(int(rel.split("ment_")[1]) - 1)]
            )
        old = sc_ref.par(parname, {"node_loc": region, "relation": rel_pre})

        # Replace relation and technology names for this bin
        curtail_tec = curt_relation_tech[rel_pre]
        old["relation"] = rel
        old.loc[old["technology"] == curtail_tec, "technology"] = curt_relation_tech[rel]

    else:
        curtail_tec = curt_relation_tech[rel]
        
    # Generate theoretical curtailment bins (with contributor technologies, but
    # these will be removed at the end)
    new = old.copy()
    # Edit the % share of wind/solar bins (Notice (-) sign)
    new.loc[new["technology"] == "elec_t_d", "value"] = -bins_dict[rel]

    # Add theoretical curtailment to the scenario
    sc.add_par(parname, new)

    # Add an upper bound for the theoretical curtailment (needed for new bins)
    bound = new.drop_duplicates(["node_rel", "year_rel", "relation"]).copy()
    bound["value"] = 0
    sc.add_par("relation_upper", bound)

    # Update the data of "input" electricity for this curtailment technology
    inp = sc.par("input", {"node_loc": region, "technology": curtail_tec})
    inp["technology"] = curt_relation_tech[rel] # is needed in case a new bin is added
    inp["value"] = 1 # Note that this is different compared to the old approach. The gamma coefficients are added later.
    sc.add_par("input", inp)
print("- New theoretical curtailment relations configured.")


- New theoretical curtailment relations configured.


Add new bins to the set of relations and new technologies to technology in the scenario:

In [188]:
sc.add_set("relation", new_bins.keys())

for rel_new, region in product(sorted(new_bins.keys()), regions):
    # Relevant wind/solar relations
    relations = [x for x, y in curt_relation_tech.items() if y in new_bins[rel_new]]

    # Load the old data of contributors and use the average of solar and wind
    # TODO: The contributions must come from PyPSA. At the moment, the old values are
    # used for illustration
    old = sc.par(parname, {"node_loc": region, 
                           "relation": relations})
    if old.empty:
        continue

    # Keep only contributor and curtailment technologies
    # This does not need VRE generation and electricity grid, as the curtailment
    # bins as % of the grid were calculated in step (3) and here we use them
    techs = [x for x in set(old["technology"]) if
             not any([y in x for y in ["wind_r", "solar_r", "elec_t_d"]])]
    
    new = old.loc[old["technology"].isin(techs)].copy()

    # Group by technology and take the sum of contribution values for solar and
    # wind for this VRE bin (e.g., if storage has 0.2 for wind, 0.1 for solar,
    # sum(0.2 , 0.1) = 0.3 will be used for this VRE bin)
    new = new.groupby(
        ["node_rel", "node_loc", "mode", "technology", "year_rel", "year_act"]
                      ).sum(numeric_only=True)
    new = new.assign(relation=rel_new, unit="GWa").reset_index()

    # insert value

    new.loc[new["technology"].isin(new_bins[rel_new]), "value"] = 1
    
    for t in curt_relation[rel_new][0].keys():
        new_ind = new.query("technology == @t").index

        if len(new_ind) > 0:
            new.loc[new_ind,"value"] = curt_relation[rel_new][0][t]
        else:
            df = make_df("relation_activity",
                        relation=rel_new,
                        node_rel="R11_WEU",
                        year_rel=years,
                        node_loc="R11_WEU",
                        technology=t,
                        year_act=years,
                        mode = "M1",
                        value = curt_relation[rel_new][0][t],
                        unit = "???")
        
        sc.add_par("relation_activity", df)

    # Add the new equation for the VRE curtailment bin to the scenario
    sc.add_par(parname, new)

    # Add an upper bound for the new relation
    bound = new.drop_duplicates(["node_rel", "year_rel", "relation"]).copy()
    bound["value"] = 0
    sc.add_par("relation_upper", bound)

    # If treating input electricity at the VRE level (combination of solar-wind bins)
    
    # Relevant VRE technology
    tech_new = ("_").join(rel_new.split("_")[:2]) + rel_new.split("_")[2]
    
    # Add new VRE technology to the scenario
    sc.add_set("technology", tech_new)
    
    # Add VRE technology to this curtailment relation
    vre = new.loc[new["technology"].isin(new_bins[rel_new])]
    
    # Change the sign of VRE curtailment value to (-)
    # VRE curtailment is equal to the unmet curtailment of wind and solar, i.e.:
    # wind_curtail + solar_curtail <= storage (all contributors) + VRE_curtail
    vre = vre.assign(technology=tech_new, 
                     value=-1)
    
    sc.add_par(parname, vre)

    # Add "input" electricity for this VRE technology
    # Load sample data and copy it for each bin
    # if include_vre_input_loss:
    inp = sc.par("input", {"node_loc": region,
                            "technology": "solar_curtailment1"})
    
    # Add input values for curtailed electricity at each combination of wind and solar
    inp = inp.assign(technology=tech_new, 
                        value=1)
    
    sc.add_par("input", inp)

print("- New VRE curtailment relations configured.")


- New VRE curtailment relations configured.


Remove the contributor technologies from theoretical relations:

In [189]:
# Since they are already added to new VRE relations, the contributor technologies are removed from the theoretical relations
for rel, region in product(sorted(bins_dict.keys()), regions):
    # Load existing data

    df = sc.par(parname, {"node_loc": region, "relation": rel})
    
    contributors = [x for x in set(df["technology"]) if
                    not any([y in x for y in ["wind", "solar", "elec_t_d"]])]
    
    df = df.loc[df["technology"].isin(contributors)].copy()
    # Remove contributors from theoretical relations
    
    sc.remove_par(parname, df)

    # Removing input electricity from wind and solar
    # inp = sc.par("input", {"node_loc": region, "technology": curt_relation_tech[rel]})
    # sc.remove_par("input", inp)

# remove "stor_ppl" since we added "LDES" and "SDES" instead
df = sc.par("relation_activity",
            {"node_loc": region, 
             "technology": "stor_ppl"})

sc.remove_par("relation_activity", df)

sc.commit("Curtailment updated")

Remove all technologies from curtailment relations

In [ ]:
df = sc.par("relation_activity",
            {"relation":"wind_curtailment_1",
             "node_loc": region, 
             "technology": "stor_ppl"})

# sc.remove_par("relation_activity", df)


### 5. Solve scenario

In [190]:
if solve_scenario:
    #case = sc.model + '__' + sc.scenario + '__v' + str(sc.version)
    print(f'Solving scenario "{scen}", please wait...!')

    start = timer()
#     sc.solve(model='MESSAGE', case=case)
    sc.solve(solve_options={"lpmethod": "4",
                            "advind":"0",
                            "lpmethod":"4","threads":"4", 
                            "epopt":"1e-06",
                            "scaind":"-1"})

    end = timer()
    print('Elapsed time for solving scenario:', int((end - start)/60),
          'min and', round((end - start) % 60, 2), 'sec.')
    sc.set_as_default()


Solving scenario "MESSAGEix-GLOBIOM__ENGAGE_SSP2_EN_NPi2020_500__v29", please wait...!
Elapsed time for solving scenario: 14 min and 13.9 sec.


In [192]:
mp.close_db()